In [ ]:
"""
PREPROCESSING NOTEBOOK
======================
This notebook handles:
1. Data loading and exploration
2. Feature engineering
3. Data cleaning
4. Geospatial analysis
5. Image validation
"""

# ============================================================================
# CELL 1: SETUP AND IMPORTS
# ============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
from config import Config
from data_fetcher import SatelliteImageFetcher

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Print configuration
Config.print_config()

# ============================================================================
# CELL 2: LOAD DATA
# ============================================================================
print("\n" + "="*70)
print("LOADING DATA")
print("="*70)

# Load train and test data
train_df = pd.read_excel(Config.TRAIN_DATA_PATH)
test_df = pd.read_excel(Config.TEST_DATA_PATH)

print(f"\nTrain shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

print("\nTrain columns:")
print(train_df.columns.tolist())

print("\nFirst few rows:")
print(train_df.head())

print("\nData types:")
print(train_df.dtypes)

print("\nBasic statistics:")
print(train_df.describe())

# ============================================================================
# CELL 3: EXPLORATORY DATA ANALYSIS - MISSING VALUES
# ============================================================================
print("\n" + "="*70)
print("MISSING VALUES ANALYSIS")
print("="*70)

missing_train = train_df.isnull().sum()
missing_train_pct = 100 * missing_train / len(train_df)

missing_df = pd.DataFrame({
    'Missing_Count': missing_train,
    'Percentage': missing_train_pct
})
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

print("\nMissing values in training data:")
print(missing_df)

# Visualize missing data
if len(missing_df) > 0:
    fig, ax = plt.subplots(figsize=(10, 6))
    missing_df['Percentage'].plot(kind='barh', ax=ax)
    ax.set_xlabel('Percentage Missing (%)')
    ax.set_title('Missing Values by Feature')
    plt.tight_layout()
    plt.show()

# ============================================================================
# CELL 4: TARGET VARIABLE ANALYSIS
# ============================================================================
print("\n" + "="*70)
print("TARGET VARIABLE ANALYSIS (PRICE)")
print("="*70)

print(f"\nPrice statistics:")
print(train_df['price'].describe())

# Visualize price distribution
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Histogram
axes[0].hist(train_df['price'], bins=50, edgecolor='black')
axes[0].set_xlabel('Price')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Price Distribution')

# Log-transformed histogram
axes[1].hist(np.log1p(train_df['price']), bins=50, edgecolor='black', color='orange')
axes[1].set_xlabel('Log(Price)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Log-Transformed Price Distribution')

# Box plot
axes[2].boxplot(train_df['price'])
axes[2].set_ylabel('Price')
axes[2].set_title('Price Box Plot')

plt.tight_layout()
plt.show()

# Check for outliers
Q1 = train_df['price'].quantile(0.25)
Q3 = train_df['price'].quantile(0.75)
IQR = Q3 - Q1
outliers = train_df[(train_df['price'] < Q1 - 1.5*IQR) | (train_df['price'] > Q3 + 1.5*IQR)]
print(f"\nNumber of price outliers (IQR method): {len(outliers)}")

# ============================================================================
# CELL 5: FEATURE CORRELATIONS
# ============================================================================
print("\n" + "="*70)
print("FEATURE CORRELATIONS")
print("="*70)

# Select numeric columns
numeric_cols = train_df.select_dtypes(include=[np.number]).columns
correlation_matrix = train_df[numeric_cols].corr()

# Correlations with price
price_correlations = correlation_matrix['price'].sort_values(ascending=False)
print("\nTop 10 features correlated with price:")
print(price_correlations.head(10))

# Visualize correlation matrix
plt.figure(figsize=(14, 12))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

# Visualize top correlations with price
top_features = price_correlations.head(11).index[1:]  # Exclude price itself
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
axes = axes.flatten()

for idx, feature in enumerate(top_features):
    axes[idx].scatter(train_df[feature], train_df['price'], alpha=0.5)
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel('Price')
    axes[idx].set_title(f'{feature} vs Price')

plt.tight_layout()
plt.show()

# ============================================================================
# CELL 6: GEOSPATIAL ANALYSIS
# ============================================================================
print("\n" + "="*70)
print("GEOSPATIAL ANALYSIS")
print("="*70)

# Check coordinate ranges
print(f"\nLatitude range: {train_df['lat'].min():.4f} to {train_df['lat'].max():.4f}")
print(f"Longitude range: {train_df['long'].min():.4f} to {train_df['long'].max():.4f}")

# Scatter plot: Geographic distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Simple scatter
axes[0].scatter(train_df['long'], train_df['lat'], alpha=0.3, s=1)
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')
axes[0].set_title('Property Locations')

# Price heatmap
scatter = axes[1].scatter(
    train_df['long'], 
    train_df['lat'], 
    c=train_df['price'], 
    cmap='YlOrRd', 
    alpha=0.5,
    s=10
)
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')
axes[1].set_title('Property Prices by Location')
plt.colorbar(scatter, ax=axes[1], label='Price')

plt.tight_layout()
plt.show()

# Price by location statistics
print("\nPrice statistics by geographic regions:")
lat_bins = pd.qcut(train_df['lat'], q=4, labels=['South', 'Mid-South', 'Mid-North', 'North'])
print(train_df.groupby(lat_bins)['price'].describe())

# ============================================================================
# CELL 7: FEATURE ENGINEERING
# ============================================================================
print("\n" + "="*70)
print("FEATURE ENGINEERING")
print("="*70)

def engineer_features(df):
    """Create new features from existing ones"""
    df = df.copy()
    
    # Age of property
    current_year = 2024
    df['age'] = current_year - df['yr_built']
    df['years_since_renovation'] = current_year - df['yr_renovated']
    df['is_renovated'] = (df['yr_renovated'] > 0).astype(int)
    
    # Size ratios
    df['living_lot_ratio'] = df['sqft_living'] / (df['sqft_lot'] + 1)
    df['above_ground_ratio'] = df['sqft_above'] / (df['sqft_living'] + 1)
    df['basement_ratio'] = df['sqft_basement'] / (df['sqft_living'] + 1)
    
    # Room ratios
    df['bath_bed_ratio'] = df['bathrooms'] / (df['bedrooms'] + 1)
    df['rooms_per_sqft'] = (df['bedrooms'] + df['bathrooms']) / (df['sqft_living'] + 1)
    
    # Neighborhood comparison
    df['living_vs_neighbors'] = df['sqft_living'] / (df['sqft_living15'] + 1)
    df['lot_vs_neighbors'] = df['sqft_lot'] / (df['sqft_lot15'] + 1)
    
    # Price per sqft (for train only)
    if 'price' in df.columns:
        df['price_per_sqft'] = df['price'] / (df['sqft_living'] + 1)
    
    # Quality scores
    df['overall_quality'] = df['grade'] * df['condition']
    df['luxury_score'] = df['grade'] + df['view'] + df['waterfront']*2
    
    # Log transformations for skewed features
    for col in ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement']:
        if col in df.columns:
            df[f'log_{col}'] = np.log1p(df[col])
    
    return df

# Apply feature engineering
train_engineered = engineer_features(train_df)
test_engineered = engineer_features(test_df)

print(f"\nOriginal features: {train_df.shape[1]}")
print(f"After engineering: {train_engineered.shape[1]}")
print(f"\nNew features added: {train_engineered.shape[1] - train_df.shape[1]}")

new_features = [col for col in train_engineered.columns if col not in train_df.columns]
print(f"\nNew feature names:")
for feat in new_features:
    print(f"  - {feat}")

# ============================================================================
# CELL 8: DOWNLOAD SATELLITE IMAGES
# ============================================================================
print("\n" + "="*70)
print("DOWNLOADING SATELLITE IMAGES")
print("="*70)

# Initialize fetcher
fetcher = SatelliteImageFetcher()

# Download images for train set
print("\nDownloading training images...")
train_with_images = fetcher.fetch_and_save_images(
    train_engineered,
    save_dir=Config.IMAGE_SAVE_DIR / 'train',
    delay=0.1
)

# Download images for test set
print("\nDownloading test images...")
test_with_images = fetcher.fetch_and_save_images(
    test_engineered,
    save_dir=Config.IMAGE_SAVE_DIR / 'test',
    delay=0.1
)

# ============================================================================
# CELL 9: VALIDATE IMAGES
# ============================================================================
print("\n" + "="*70)
print("IMAGE VALIDATION")
print("="*70)

from PIL import Image

def validate_images(df, sample_size=5):
    """Validate and visualize sample images"""
    valid_images = df[df['image_path'].notna()]
    
    print(f"\nValid images: {len(valid_images)}/{len(df)}")
    
    # Display sample images
    sample = valid_images.sample(min(sample_size, len(valid_images)))
    
    fig, axes = plt.subplots(1, len(sample), figsize=(20, 4))
    if len(sample) == 1:
        axes = [axes]
    
    for idx, (_, row) in enumerate(sample.iterrows()):
        img = Image.open(row['image_path'])
        axes[idx].imshow(img)
        axes[idx].axis('off')
        if 'price' in row:
            axes[idx].set_title(f"Price: ${row['price']:,.0f}")
        else:
            axes[idx].set_title(f"ID: {row['id']}")
    
    plt.tight_layout()
    plt.show()

# Validate train images
print("\nSample training images:")
validate_images(train_with_images, sample_size=5)

# Validate test images
print("\nSample test images:")
validate_images(test_with_images, sample_size=5)

# ============================================================================
# CELL 10: SAVE PROCESSED DATA
# ============================================================================
print("\n" + "="*70)
print("SAVING PROCESSED DATA")
print("="*70)

# Save processed datasets
output_dir = Config.DATA_DIR / 'processed'
output_dir.mkdir(parents=True, exist_ok=True)

train_output = output_dir / 'train_processed.csv'
test_output = output_dir / 'test_processed.csv'

train_with_images.to_csv(train_output, index=False)
test_with_images.to_csv(test_output, index=False)

print(f"\n✓ Saved processed training data to {train_output}")
print(f"✓ Saved processed test data to {test_output}")

print("\n" + "="*70)
print("PREPROCESSING COMPLETE")
print("="*70)
print("\nNext steps:")
print("1. Review the generated visualizations")
print("2. Check the processed data files")
print("3. Proceed to model_training.ipynb")